# Exploratory Notebook

# Business Understanding

## Business Context
Our company is entering the competitive movie production industry by establishing a new movie studio. With major corporations increasingly investing in original video content, we need to strategically position ourselves to maximize our chances of box office success.

## Primary Stakeholder
**Head of New Movie Studio** - The key decision-maker who will use our analysis to guide the studio's content creation strategy and investment decisions.

## Key Business Questions

### 1. **What genres are most profitable?**
   - Which movie genres consistently generate the highest box office returns?
   - Are there emerging genres showing strong growth potential?

### 2. **What budget ranges optimize ROI?**
   - What is the relationship between production budget and box office performance?
   - What budget sweet spot maximizes return on investment?

### 3. **When should we release our films?**
   - Which months/seasons show the strongest box office performance?
   - How can we avoid oversaturated release periods?

## Business Objectives
- **Primary Goal:** Identify the most profitable film characteristics to guide our initial movie productions
- **Success Metrics:** Maximize box office revenue and return on investment
- **Risk Mitigation:** Minimize financial losses by avoiding poorly-performing film categories

## Data Requirements & Analysis Considerations

### Dataset Specifications Needed:
1. **What movie data do we have access to?** (Box office numbers, budgets, genres, release dates, ratings, etc.)
2. **What time period does our data cover?** (Recent years only, or historical data?)
3. **Is it domestic box office, international, or worldwide gross?**

### Business Context Clarifications:
4. **What size studio are we targeting?** (Independent/small budget vs. major studio competitor?)
5. **Are there any genre restrictions?** (Family-friendly only, or open to all ratings?)
6. **Geographic focus?** (Domestic US market, international, or global?)

### Analysis Scope Considerations:
7. **Should we consider streaming/digital revenue or just theatrical box office?**
8. **Do we want to analyze competitor studios' strategies?**
9. **Should seasonal trends consider COVID-19 impact on recent years?**

## Success Metrics Framework
**Primary Metrics:**
- **Gross Revenue:** Total box office performance
- **Return on Investment (ROI):** (Revenue - Budget) / Budget × 100
- **Profit Margins:** Net profit as percentage of revenue
- **Budget Efficiency:** Revenue per dollar invested

**Secondary Metrics:**
- **Market Share:** Performance relative to industry
- **Risk Assessment:** Variance in returns by category
- **Audience Reach:** Ticket sales and demographic penetration

## Recommended Visualizations Strategy

### 1. **Genre Analysis**
- **Horizontal Bar Chart:** Average revenue by genre (easy comparison)
- **Box Plot:** Revenue distribution by genre (shows variability and outliers)
- **Scatter Plot:** Budget vs. Revenue by genre (profitability insights)

### 2. **Budget Optimization**
- **Scatter Plot with Trend Line:** Budget vs. ROI (sweet spot identification)
- **Histogram:** Distribution of profitable budget ranges
- **Bubble Chart:** Budget vs. Revenue (bubble size = profit margin)

### 3. **Release Timing**
- **Line Graph:** Monthly box office performance over time
- **Heatmap:** Revenue by month and genre combination
- **Seasonal Bar Chart:** Quarterly performance comparison

### 4. **Risk vs. Reward Analysis**
- **Risk-Return Scatter Plot:** Average return vs. standard deviation by category
- **Waterfall Chart:** Success rate breakdown by film characteristics

## Expected Deliverables
Three concrete, data-driven recommendations that the Head of Movie Studio can immediately implement in the studio's content strategy and production planning.

# Data Understanding

The `bom.movie_gross.csv.gz` contains the following columns:


In [50]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from random import gauss
from scipy import stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from mpl_toolkits import mplot3d
import sklearn.metrics as metrics
import statsmodels.api as sm
import sqlite3

In [51]:
path_1 = ("../Data/bom.movie_gross.csv")
path_2 = ("../Data/rt.movie_info.tsv")
path_3 = ("../Data/rt.reviews.tsv")
path_4 = ("../Data/tmdb.movies.csv")
path_5 = ("../Data/tn.movie_budgets.csv")

In [52]:

bom_movie = pd.read_csv(path_1)
rt_info = pd.read_csv(path_2, sep='\t', encoding='latin1')


rt_reviews = pd.read_csv(path_3, sep='\t', encoding='latin1')

tmdb_df = pd.read_csv(path_4)
tn_budget = pd.read_csv(path_5)

In [53]:
#going through SQl
def load_tables(connection):
    table_names = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", connection
    )["name"].tolist()
    sql_tables = {
        table_name: pd.read_sql(f"SELECT * FROM {table_name}", connection)
        for table_name in table_names
    }
    return sql_tables

path = '../Data/im.db'
conn = sqlite3.connect(path)

imdb_tables = load_tables(conn)

print(imdb_tables.keys())


dict_keys(['movie_basics', 'directors', 'known_for', 'movie_akas', 'movie_ratings', 'persons', 'principals', 'writers'])


In [54]:
directors = imdb_tables['directors']
directors.info()
directors.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291174 entries, 0 to 291173
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   291174 non-null  object
 1   person_id  291174 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


movie_id     0
person_id    0
dtype: int64

In [55]:
directors_clean = imdb_tables['directors'].copy(deep=True)

before = directors_clean.shape[0]
directors_clean.drop_duplicates(keep='first', inplace=True)
after = directors_clean.shape[0]

print(f"Dropped {before - after} duplicate rows.")
print(directors_clean.shape)


Dropped 127639 duplicate rows.
(163535, 2)


In [56]:
known_for = imdb_tables['known_for']
print(known_for.info())
print(known_for.isna().sum())
print(known_for.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638260 entries, 0 to 1638259
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   person_id  1638260 non-null  object
 1   movie_id   1638260 non-null  object
dtypes: object(2)
memory usage: 25.0+ MB
None
person_id    0
movie_id     0
dtype: int64
(1638260, 2)


In [57]:
known_for_clean = imdb_tables['known_for'].copy(deep=True)
before = known_for_clean.shape[0]
known_for_clean.drop_duplicates(keep='first', inplace=True)
after = known_for_clean.shape[0]
print(f"Dropped {before - after} duplicate rows.")
print(known_for_clean.shape)

Dropped 0 duplicate rows.
(1638260, 2)


In [58]:
movie_akas = imdb_tables['movie_akas']
print(movie_akas.info())
print(movie_akas.shape)
print(movie_akas.isna().any())
print(movie_akas.isna().sum())

movie_akas


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           331703 non-null  object 
 1   ordering           331703 non-null  int64  
 2   title              331703 non-null  object 
 3   region             278410 non-null  object 
 4   language           41715 non-null   object 
 5   types              168447 non-null  object 
 6   attributes         14925 non-null   object 
 7   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB
None
(331703, 8)
movie_id             False
ordering             False
title                False
region                True
language              True
types                 True
attributes            True
is_original_title     True
dtype: bool
movie_id                  0
ordering                  0
title                     0
region  

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [59]:
movie_akas_clean = imdb_tables['movie_akas'].copy(deep=True)
#filling missing regions with unknown
movie_akas_clean['region'] = movie_akas_clean['region'].fillna("UNK")
#due to high missingness, drop the language table, the  types column too, attribiutes too, is_original also goes down
movie_akas_clean = movie_akas_clean.drop(['language','types', 'attributes', 'is_original_title'], axis=1)
movie_akas_clean['region'].unique()
region_map = {
    'AD': 'Andorra',
    'AE': 'United Arab Emirates',
    'AF': 'Afghanistan',
    'AG': 'Antigua and Barbuda',
    'AI': 'Anguilla',
    'AL': 'Albania',
    'AM': 'Armenia',
    'AN': 'Netherlands Antilles (former)',
    'AO': 'Angola',
    'AQ': 'Antarctica',
    'AR': 'Argentina',
    'AS': 'American Samoa',
    'AT': 'Austria',
    'AU': 'Australia',
    'AW': 'Aruba',
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herzegovina',
    'BB': 'Barbados',
    'BD': 'Bangladesh',
    'BE': 'Belgium',
    'BF': 'Burkina Faso',
    'BG': 'Bulgaria',
    'BH': 'Bahrain',
    'BI': 'Burundi',
    'BJ': 'Benin',
    'BM': 'Bermuda',
    'BN': 'Brunei Darussalam',
    'BO': 'Bolivia',
    'BR': 'Brazil',
    'BS': 'Bahamas',
    'BT': 'Bhutan',
    'BUMM': 'Burma (former name for Myanmar)',
    'BY': 'Belarus',
    'BZ': 'Belize',
    'CA': 'Canada',
    'CD': 'Congo, Democratic Republic of the',
    'CF': 'Central African Republic',
    'CG': 'Congo, Republic of the',
    'CH': 'Switzerland',
    'CI': 'Côte d\'Ivoire',
    'CL': 'Chile',
    'CM': 'Cameroon',
    'CN': 'China',
    'CO': 'Colombia',
    'CR': 'Costa Rica',
    'CSHH': 'Czechoslovakia (former)',
    'CSXX': 'Serbia and Montenegro (former)',
    'CU': 'Cuba',
    'CV': 'Cabo Verde',
    'CY': 'Cyprus',
    'CZ': 'Czechia',
    'DE': 'Germany',
    'DK': 'Denmark',
    'DM': 'Dominica',
    'DO': 'Dominican Republic',
    'DZ': 'Algeria',
    'EC': 'Ecuador',
    'EE': 'Estonia',
    'EG': 'Egypt',
    'EH': 'Western Sahara',
    'ER': 'Eritrea',
    'ES': 'Spain',
    'ET': 'Ethiopia',
    'FI': 'Finland',
    'FJ': 'Fiji',
    'FO': 'Faroe Islands',
    'FR': 'France',
    'GA': 'Gabon',
    'GB': 'United Kingdom',
    'GE': 'Georgia',
    'GH': 'Ghana',
    'GL': 'Greenland',
    'GM': 'Gambia',
    'GP': 'Guadeloupe',
    'GR': 'Greece',
    'GT': 'Guatemala',
    'GU': 'Guam',
    'GW': 'Guinea-Bissau',
    'HK': 'Hong Kong',
    'HN': 'Honduras',
    'HR': 'Croatia',
    'HT': 'Haiti',
    'HU': 'Hungary',
    'ID': 'Indonesia',
    'IE': 'Ireland',
    'IL': 'Israel',
    'IM': 'Isle of Man',
    'IN': 'India',
    'IQ': 'Iraq',
    'IR': 'Iran',
    'IS': 'Iceland',
    'IT': 'Italy',
    'JM': 'Jamaica',
    'JO': 'Jordan',
    'JP': 'Japan',
    'KE': 'Kenya',
    'KG': 'Kyrgyzstan',
    'KH': 'Cambodia',
    'KN': 'Saint Kitts and Nevis',
    'KP': 'North Korea',
    'KR': 'South Korea',
    'KW': 'Kuwait',
    'KY': 'Cayman Islands',
    'KZ': 'Kazakhstan',
    'LA': 'Laos',
    'LB': 'Lebanon',
    'LC': 'Saint Lucia',
    'LI': 'Liechtenstein',
    'LK': 'Sri Lanka',
    'LR': 'Liberia',
    'LS': 'Lesotho',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'LY': 'Libya',
    'MA': 'Morocco',
    'MC': 'Monaco',
    'MD': 'Moldova',
    'ME': 'Montenegro',
    'MG': 'Madagascar',
    'MH': 'Marshall Islands',
    'MK': 'North Macedonia',
    'ML': 'Mali',
    'MM': 'Myanmar',
    'MN': 'Mongolia',
    'MO': 'Macao',
    'MQ': 'Martinique',
    'MR': 'Mauritania',
    'MT': 'Malta',
    'MU': 'Mauritius',
    'MV': 'Maldives',
    'MW': 'Malawi',
    'MX': 'Mexico',
    'MY': 'Malaysia',
    'MZ': 'Mozambique',
    'NC': 'New Caledonia',
    'NE': 'Niger',
    'NG': 'Nigeria',
    'NI': 'Nicaragua',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'NP': 'Nepal',
    'NZ': 'New Zealand',
    'OM': 'Oman',
    'PA': 'Panama',
    'PE': 'Peru',
    'PF': 'French Polynesia',
    'PG': 'Papua New Guinea',
    'PH': 'Philippines',
    'PK': 'Pakistan',
    'PL': 'Poland',
    'PR': 'Puerto Rico',
    'PS': 'Palestine, State of',
    'PT': 'Portugal',
    'PY': 'Paraguay',
    'QA': 'Qatar',
    'RE': 'Réunion',
    'RO': 'Romania',
    'RS': 'Serbia',
    'RU': 'Russia',
    'RW': 'Rwanda',
    'SA': 'Saudi Arabia',
    'SB': 'Solomon Islands',
    'SD': 'Sudan',
    'SE': 'Sweden',
    'SG': 'Singapore',
    'SI': 'Slovenia',
    'SK': 'Slovakia',
    'SL': 'Sierra Leone',
    'SM': 'San Marino',
    'SN': 'Senegal',
    'SO': 'Somalia',
    'SR': 'Suriname',
    'SUHH': 'Soviet Union (former)',
    'SV': 'El Salvador',
    'SY': 'Syrian Arab Republic',
    'SZ': 'Eswatini',
    'TD': 'Chad',
    'TG': 'Togo',
    'TH': 'Thailand',
    'TJ': 'Tajikistan',
    'TL': 'Timor-Leste',
    'TM': 'Turkmenistan',
    'TN': 'Tunisia',
    'TO': 'Tonga',
    'TR': 'Turkey',
    'TT': 'Trinidad and Tobago',
    'TW': 'Taiwan',
    'TZ': 'Tanzania',
    'UA': 'Ukraine',
    'UG': 'Uganda',
    'UNK': 'Unknown',
    'US': 'United States',
    'UY': 'Uruguay',
    'UZ': 'Uzbekistan',
    'VC': 'Saint Vincent and the Grenadines',
    'VE': 'Venezuela',
    'VI': 'Virgin Islands, U.S.',
    'VN': 'Vietnam',
    'VU': 'Vanuatu',
    'WF': 'Wallis and Futuna',
    'XAS': 'Southeast Asia (unofficial)',
    'XEU': 'Europe (unofficial)',
    'XKO': 'Kosovo (unofficial)',
    'XKV': 'Kosovo (under UNSC resolution)',
    'XNA': 'North America (unofficial)',
    'XSA': 'South Asia (unofficial)',
    'XWG': 'West Germany (former)',
    'XWW': 'World-wide (English title)',
    'XYU': 'Yugoslavia (former)',
    'YE': 'Yemen',
    'ZA': 'South Africa',
    'ZM': 'Zambia',
    'ZW': 'Zimbabwe'
}
movie_akas_clean['region_full'] = movie_akas_clean['region'].map(region_map).str.upper()
movie_akas_clean.duplicated().sum()

movie_akas_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   movie_id     331703 non-null  object
 1   ordering     331703 non-null  int64 
 2   title        331703 non-null  object
 3   region       331703 non-null  object
 4   region_full  331703 non-null  object
dtypes: int64(1), object(4)
memory usage: 12.7+ MB


In [60]:
writers = imdb_tables['writers']
writers.info()
writers.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255873 entries, 0 to 255872
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   255873 non-null  object
 1   person_id  255873 non-null  object
dtypes: object(2)
memory usage: 3.9+ MB


(255873, 2)

In [61]:
writers_clean = imdb_tables['writers'].copy(deep=True)
before = writers_clean.shape[0]
writers_clean.drop_duplicates(keep='first', inplace=True)
after = writers_clean.shape[0]
print(f"Dropped {before - after} duplicate rows.")
print(writers_clean.shape)

Dropped 77521 duplicate rows.
(178352, 2)
